<a href="https://colab.research.google.com/github/sda96/AIFFEL_3rd_hackerton_TUNiB_DKTC/blob/main/notebook/SeungTaek/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,887 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155229 files and d

In [1]:
!gdown https://drive.google.com/uc?id=1A8IdrqgEjRhvg9KoV6dopo5GirTKO3aj
!gdown https://drive.google.com/uc?id=1SCxtUqlGEzYCfCbHUREFf5UqoM1uWDBK
!gdown https://drive.google.com/uc?id=1Zh4mCpgQReE4E0sAjiFXwSV6Y7lc77fO
!gdown https://drive.google.com/uc?id=19jT2ZTb6oj6sFERkl5AT3qQExxN3D5cK

Downloading...
From: https://drive.google.com/uc?id=1A8IdrqgEjRhvg9KoV6dopo5GirTKO3aj
To: /content/sns_val.zip
100% 34.8M/34.8M [00:01<00:00, 34.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SCxtUqlGEzYCfCbHUREFf5UqoM1uWDBK
To: /content/sns_train.zip
100% 278M/278M [00:04<00:00, 67.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Zh4mCpgQReE4E0sAjiFXwSV6Y7lc77fO
To: /content/train.csv
100% 2.24M/2.24M [00:00<00:00, 19.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=19jT2ZTb6oj6sFERkl5AT3qQExxN3D5cK
To: /content/test.json
100% 417k/417k [00:00<00:00, 6.41MB/s]


In [2]:
!unzip -uq "/content/sns_train.zip" -d "/content/sns_data"
!unzip -uq "/content/sns_val.zip" -d "/content/sns_data"

In [47]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 31.0 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 6.8 MB 59.9 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [50]:
import pandas as pd # pandas 모듈 로드
import json  # json 모듈 로드
import re
import os
from glob import glob
from tqdm import tqdm
from transformers import TFBertForSequenceClassification, BertTokenizer
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer, AutoModel

In [27]:
train_df = pd.read_csv("/content/train.csv")

In [28]:
train_df

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [8]:
data_path ="/content/sns_data/content/sns_csv_train"
files = glob(data_path + "/*")
files

['/content/sns_data/content/sns_csv_train/상거래(쇼핑).csv',
 '/content/sns_data/content/sns_csv_train/주거와생활.csv',
 '/content/sns_data/content/sns_csv_train/미용과건강.csv',
 '/content/sns_data/content/sns_csv_train/일과직업.csv',
 '/content/sns_data/content/sns_csv_train/행사.csv',
 '/content/sns_data/content/sns_csv_train/식음료.csv',
 '/content/sns_data/content/sns_csv_train/시사교육.csv',
 '/content/sns_data/content/sns_csv_train/개인및관계.csv',
 '/content/sns_data/content/sns_csv_train/여가생활.csv']

In [10]:
df_all = pd.DataFrame()
for i in files:
    df = pd.read_csv(i)
    df_all = pd.concat([df_all, df])

In [11]:
df_all

,0
0,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...
1,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...
2,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...
3,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...
4,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...
...,...
190310,그래서 나는 국대경기를 젤 좋아해\n나 무슨 마음인지\n내가 제일 좋아하는 선수들만...
190311,. .\n좋은겅가ㅎㅎㅎㅎㅎㅎㅎㅎ\n그래. . 나한테만하지말고 다른데도 좀 써먹어. ...
190312,근데 요즘 비둘기들 진짜 겁대가리상실\nㅋㅋㅋㅋㅋㅋㅋ겁대가리 상실ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
190313,#@계정# 토스 퀴즈 정답 0825\n오올~~~ 141원 들어왔어요^^\n오오 ㅋ...


In [12]:
df_all["class"] = "일반 대화"

In [13]:
df_all.head()

,0,class
0,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...,일반 대화
1,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...,일반 대화
2,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...,일반 대화
3,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...,일반 대화
4,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...,일반 대화


In [20]:
df_all = df_all.rename(columns = {'0' : "conversation"})

In [21]:
df_all

,conversation,class
0,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...,일반 대화
1,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...,일반 대화
2,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...,일반 대화
3,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...,일반 대화
4,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...,일반 대화
...,...,...
190310,그래서 나는 국대경기를 젤 좋아해\n나 무슨 마음인지\n내가 제일 좋아하는 선수들만...,일반 대화
190311,. .\n좋은겅가ㅎㅎㅎㅎㅎㅎㅎㅎ\n그래. . 나한테만하지말고 다른데도 좀 써먹어. ...,일반 대화
190312,근데 요즘 비둘기들 진짜 겁대가리상실\nㅋㅋㅋㅋㅋㅋㅋ겁대가리 상실ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,일반 대화
190313,#@계정# 토스 퀴즈 정답 0825\n오올~~~ 141원 들어왔어요^^\n오오 ㅋ...,일반 대화


In [23]:
df_all = df_all[['class', 'conversation']]
df_all

,class,conversation
0,일반 대화,맥주삿당 역시 태풍엔 맥주지\n오 뭐뭐 샀음?\n필라이트 좡색\n오 나는 구스 아일...
1,일반 대화,일단 니 에스쁘아쿠션까지해서 총6개\n에스쁘아쿠션사게?̊̈ ㅋㅋㅋㅋㅋ\n아니 니 어...
2,일반 대화,누리카드사용 낼부터된다네 책을 좀 빨리봐야해서 오늘 그냥주문해줘\n알겠음 집으로 시...
3,일반 대화,언니 텐텐서 컵 살거야? 살 거면 나 테이블보? 무슨 패브릭 이런거 하나만 사게 업...
4,일반 대화,하 어쩌냐 팔기싫다 또 질문해 어찌고저찌고\nㅋㅋㅋㅋㅋㅋㅋ 아 또 뭐래...\n블투...
...,...,...
190310,일반 대화,그래서 나는 국대경기를 젤 좋아해\n나 무슨 마음인지\n내가 제일 좋아하는 선수들만...
190311,일반 대화,. .\n좋은겅가ㅎㅎㅎㅎㅎㅎㅎㅎ\n그래. . 나한테만하지말고 다른데도 좀 써먹어. ...
190312,일반 대화,근데 요즘 비둘기들 진짜 겁대가리상실\nㅋㅋㅋㅋㅋㅋㅋ겁대가리 상실ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
190313,일반 대화,#@계정# 토스 퀴즈 정답 0825\n오올~~~ 141원 들어왔어요^^\n오오 ㅋ...


In [25]:
df_train2 = df_all.sample(n = 900, random_state = 5)

In [41]:
df_train = pd.concat([train_df, df_train2])
df_train

,idx,class,conversation
0,0.0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1.0,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2.0,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3.0,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4.0,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
77521,NaN,일반 대화,그거먹고 말하면안대 내가 대화해바써 먹은분이랑\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
171831,NaN,일반 대화,강아지 미용해가지고 몸은 날렵한데 꼬리는 풍성해\n진짜 치사하다 혼자만 보고\n사진...
138889,NaN,일반 대화,오잉\n#@시스템#사진#\n#@이름# 잘때 다 그릿네 잘그릿단\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
85660,NaN,일반 대화,이거 5회 대면 5회가 최대 지원 횟수래\nㅋㅋ 일단 4회지원해놈\nㅋㅋㅋ잘했다 시...


In [42]:
df_train = df_train.drop(['idx'],axis=1)
df_train

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
77521,일반 대화,그거먹고 말하면안대 내가 대화해바써 먹은분이랑\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
171831,일반 대화,강아지 미용해가지고 몸은 날렵한데 꼬리는 풍성해\n진짜 치사하다 혼자만 보고\n사진...
138889,일반 대화,오잉\n#@시스템#사진#\n#@이름# 잘때 다 그릿네 잘그릿단\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...
85660,일반 대화,이거 5회 대면 5회가 최대 지원 횟수래\nㅋㅋ 일단 4회지원해놈\nㅋㅋㅋ잘했다 시...


In [43]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,class,conversation
0,기타 괴롭힘 대화,야 오늘 체격검사 한 결과 봤냐? 결과 바봐 개웃김\n야 키랑 몸무게가 왜 이래 사...
1,갈취 대화,학생회비로 3만원씩 000으러 입금해주세요\n보냈습니다.\n저도 보냈습니다.\n이건...
2,기타 괴롭힘 대화,너 언니 잘나간다고 깝치는것도 정도껏해 언니 믿고 나대지말라고\n뭐?\n너는 아무것...
3,기타 괴롭힘 대화,야 너 진짜 못생겼다\n네?\n얼굴이 그게 뭐니?\n초면에 뭐하시는거죠\n초면이고 ...
4,기타 괴롭힘 대화,야 너 왜그렇게 걷냐?\n하지마.\n너 장애인이냐?\n그만해.\n니 부모님도 장애인...
...,...,...
4845,일반 대화,새우볶음밥 먹고싶어\n#@시스템#검색#\n요거 핫템인가?ㅋㅋㅋㅋ\n어 이거맛잇을거같...
4846,갈취 대화,야 너 돈좀 줘야 될것 같다\n내가 왜\n뭐 이게 내가 만만하나 이 칼 안보여? 새...
4847,협박 대화,이게 어떻게 된거야.? 엄마 수술은 어떻게 된거냐고\n 엄마가 이제 괜찮다고 해서 ...
4848,직장 내 괴롭힘 대화,야 김대리 일로 와 봐. 이거 네가 쓴 보고서 맞지?\n 네.\n 머리 좀 대 봐....


In [40]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣0-9?.!,\n]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [44]:
df_train["conversation"] = df_train["conversation"].apply(preprocess_sentence)
df_train

,class,conversation
0,기타 괴롭힘 대화,야 오늘 체격검사 한 결과 봤냐 ? 결과 바봐 개웃김\n야 키랑 몸무게가 왜 이래 ...
1,갈취 대화,학생회비로 3만원씩 000으러 입금해주세요\n보냈습니다 . \n저도 보냈습니다 . ...
2,기타 괴롭힘 대화,너 언니 잘나간다고 깝치는것도 정도껏해 언니 믿고 나대지말라고\n뭐 ? \n너는 아...
3,기타 괴롭힘 대화,야 너 진짜 못생겼다\n네 ? \n얼굴이 그게 뭐니 ? \n초면에 뭐하시는거죠\n초...
4,기타 괴롭힘 대화,야 너 왜그렇게 걷냐 ? \n하지마 . \n너 장애인이냐 ? \n그만해 . \n니 ...
...,...,...
4845,일반 대화,새우볶음밥 먹고싶어\n 시스템 검색 \n요거 핫템인가 ? ㅋㅋㅋㅋ\n어 이거맛잇을거...
4846,갈취 대화,야 너 돈좀 줘야 될것 같다\n내가 왜\n뭐 이게 내가 만만하나 이 칼 안보여 ? ...
4847,협박 대화,이게 어떻게 된거야 . ? 엄마 수술은 어떻게 된거냐고\n 엄마가 이제 괜찮다고 해...
4848,직장 내 괴롭힘 대화,야 김대리 일로 와 봐 . 이거 네가 쓴 보고서 맞지 ? \n 네 . \n 머리 좀...


In [51]:
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")

model = AutoModel.from_pretrained("skt/kobert-base-v1")

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

ValueError: ignored